In [10]:
import sys
import os
sys.path.append(os.path.abspath('..'))
from bs4 import BeautifulSoup
from datetime import datetime
import time
import requests

In [4]:
from download_minutes import download_minutes_from_agenda


In [5]:
download_minutes_from_agenda(agenda_input_dir="../agenda", agenda_fname="2024-06-24.City-Council-Meeting.html", minutes_output_dir="test_minutes")

In [12]:
agenda_fname = "2024-06-24.City-Council-Meeting.html"
agenda_input_dir = "../agenda"
minutes_output_dir = "test_minutes"

meeting_type_str = agenda_fname.split(".")[1]
f = open(f"{agenda_input_dir}/{agenda_fname}", "r")
agenda_text = f.read()
f.close()
soup = BeautifulSoup(agenda_text, "html.parser")
attachment_divs = soup.find_all("div", attrs={"class": "AgendaItemAttachment"})


In [15]:
attachment_divs[:2]

[<div class="AgendaItemAttachment AgendaItemAttachment43"><span class="Sequence">1.</span><a class="Link" data-container="body" data-html="true" data-original-title="06-03-2024 - Special Council Minutes.pdf" data-toggle="tooltip" href="filestream.ashx?DocumentId=38115" tabindex="0" target="_blank"><span class="Link">06-03-2024 - Special Council Minutes.pdf</span></a></div>,
 <div class="AgendaItemAttachment AgendaItemAttachment44"><span class="Sequence">1.</span><a class="Link" data-container="body" data-html="true" data-original-title="06-10-2024 - Council Minutes.pdf" data-toggle="tooltip" href="filestream.ashx?DocumentId=38116" tabindex="0" target="_blank"><span class="Link">06-10-2024 - Council Minutes.pdf</span></a></div>]

In [ ]:
for attachment in attachment_divs[:1]:
    a = attachment.find("a")
    if not a:
        continue
    href = a.attrs["href"]
    url = f"https://pub-peterborough.escribemeetings.com/{href}"

    if "minutes" in a.text.lower():
        try:
            possible_date = a.text.split(" - ")[1].split(" ")[-1]
            date = datetime.strptime(possible_date, "%m-%d-%Y")
        except:
            try:
                # filename like "06-03-2024 - Special Council Minutes.pdf"
                possible_date = a.text.split(" - ")[0]
                date = datetime.strptime(possible_date, "%m-%d-%Y")
            except:
                continue
    else:
        possible_date = a.text[:10]
        if possible_date.count("-") != 2:
            continue

        if " " in possible_date:
            possible_short_date = possible_date.split(" ")[0]
            try:
                date = datetime.strptime(possible_short_date, "%m-%d-%y")
            except:
                continue
        else:
            try:
                date = datetime.strptime(possible_date, "%m-%d-%Y")
            except ValueError:
                try:
                    date = datetime.strptime(possible_date, "%Y-%m-%d")
                except:
                    continue

    date_str = datetime.strftime(date, "%Y-%m-%d")
        
    new_fname = f"{date_str}.{meeting_type_str}.pdf"
    new_fpath = f"{minutes_output_dir}/{new_fname}"

    # do not overwrite
    if os.path.isfile(new_fpath):
        continue

    # download
    resp = requests.get(url)

    with open(new_fpath, "wb") as f:
        f.write(resp.content)
    print(f"download and save {new_fpath}")

In [21]:
a.text

'06-03-2024 - Special Council Minutes.pdf'

In [22]:
possible_date

'Minutes.pdf'

In [20]:
a.attrs["href"]


'filestream.ashx?DocumentId=38115'

In [25]:
possible_date = a.text.split(" - ")[0]
date = datetime.strptime(possible_date, "%m-%d-%Y")

In [26]:
date

datetime.datetime(2024, 6, 3, 0, 0)